<a href="https://colab.research.google.com/github/DeepaliVerma/personal_files/blob/main/Triplet_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/mwray/Joint-Part-of-Speech-Embeddings/blob/c61eaabcda68051782e417e605f35163256c50a6/src/datasets/__init__.py#L11

In [2]:
def to_tensor(arr, gpu=False):
    tensor = th.tensor(arr).float()
    if gpu:
        return tensor.cuda()
    return tensor

In [4]:

  
import os
import sys
import numpy as np
import torch as th
import torch.nn as nn
import torch.nn.functional as F
# from datasets import to_tensor

class TripletLoss(nn.Module):
    """
    Basic triplet loss which calculates the distance between an query and its
    positive and negative examples.
    """
    def __init__(self, margin, weight, reduction='mean'):
        """
        Inputs:
        - margin: value to use as the margin (lower bound between positive and
          negative distances).
        - weight: the weight for this triplet loss.
        - reduction: ['mean', 'sum', 'none'] what reduction to use on the final
          loss calculation.
        """
        super(TripletLoss, self).__init__()
        self.margin = margin
        self.weight = weight
        assert reduction in {'mean', 'sum', 'none'}
        if reduction == 'mean':
            self.reduction = lambda x: x.mean()
        elif reduction == 'sum':
            self.reduction = lambda x: x.sum()
        elif reduction == 'none':
            self.reduction = lambda x: x

    def forward(self, x, pos, neg):
        d_pos = (x - pos).pow(2).sum(1)
        d_neg = (x - neg).pow(2).sum(1)
        losses =  F.relu(self.margin + d_pos - d_neg)
        return self.weight * self.reduction(losses)

if __name__ == '__main__':
    triplet_loss_m = TripletLoss(0.1, 1.0)
    triplet_loss_m_0_1 = TripletLoss(0.1, 0.1)
    triplet_loss_s = TripletLoss(0.1, 1.0, 'sum')
    triplet_loss_n = TripletLoss(0.1, 1.0, 'none')

  
    xs = to_tensor(np.random.rand(64, 256))
    pos = to_tensor(np.random.rand(64, 256))
    neg = to_tensor(np.random.rand(64, 256))

    loss_m = triplet_loss_m(xs, pos, neg)
    print('loss_m', loss_m)
    loss_m_0_1 = triplet_loss_m_0_1(xs, pos, neg)
    print('loss_m_0_1', loss_m_0_1)
    loss_s = triplet_loss_s(xs, pos, neg)
    print('loss_s', loss_s)
    loss_n = triplet_loss_n(xs, pos, neg)
    print('loss_n', loss_n)

    assert 0.1 * loss_m == loss_m_0_1

    assert loss_n.mean() == loss_m
    assert loss_n.sum() == loss_s

    assert loss_m.shape == loss_s.shape
    assert loss_n.shape == th.Size([64])
    assert loss_m.shape == th.Size([])

loss_m tensor(2.4947)
loss_m_0_1 tensor(0.2495)
loss_s tensor(159.6580)
loss_n tensor([ 0.0000,  5.8548,  4.6150,  5.7829,  0.0000,  0.0000,  7.1419,  0.3733,
         4.5315,  7.1971,  0.0000,  7.4257,  2.2678,  0.0000,  0.5463,  0.0000,
         0.0000,  0.0000,  2.9221,  4.4218,  0.0000,  0.5164,  5.9862,  5.0754,
         0.0000,  3.4822,  1.4580,  0.0000,  3.5472,  0.0000,  0.0000,  6.4927,
         6.4324,  0.0000,  4.0880,  0.0000,  9.4118,  6.7432,  0.0000,  0.2434,
         4.2728,  0.0000,  2.0297,  0.0000,  0.0000,  8.1881, 10.1981,  4.7316,
         2.0447,  1.6367,  2.5601,  0.0000,  6.7503,  0.0530,  0.0000,  0.0000,
         1.8621,  0.0000,  0.0000,  0.0000,  0.0000,  4.7693,  4.0045,  0.0000])


In [ ]:
# extra functions
def sample_triplets(anchor_idxs, x_to_class_dict, class_to_y_dict, num_triplets, sampling_method='random'):
    assert sampling_method in {'random'}
    if sampling_method == 'random':
        return sample_random_triplets(anchor_idxs, x_to_class_dict, class_to_y_dict, num_triplets)

def sample_random_triplets(anchor_idxs, x_to_class_dict, class_to_y_dict, num_triplets):
    classes = set(class_to_y_dict.keys())
    from tqdm import tqdm
    pos_idxs = []
    neg_idxs = []
    for anchor in tqdm(anchor_idxs):
        pos_class = x_to_class_dict[anchor]
        pos_idxs.append(sample_n(class_to_y_dict, [pos_class], num_triplets))
        neg_classes = list(classes - set([pos_class]))
        neg_idxs.append(sample_n(class_to_y_dict, neg_classes, num_triplets))
    pos_idxs = np.array(pos_idxs)
    neg_idxs = np.array(neg_idxs)
    return pos_idxs, neg_idxs